# Analise de Dados

**Premissa**: Execute consultas exploratórias para entender melhor os dados e validar as transformações.
Crie visualizações ou relatórios para comunicar insights.

# Tabelas

  - **olist_orders_dataset** : Tabela principal, usada para conectar todos os detalhes relacionados a um pedido de compra.
  - **olist_order_items_dataset** : Contém os detalhes de um item que foi comprado.
  - **olist_order_reviews_dataset** : Contém detalhes relacionados a quaisquer comentários postados pelo cliente sobre um produto.
  - **olist_products_dataset** : Contém informações relacionadas a um produto.
  - **olist_order_payments_dataset** : Contém informações relacionadas aos detalhes de pagamento.
  - **olist_customers_dataset** : Contém informações de clientes.
  - **olist_geolocation_dataset** : Contém informações geográficas relacionadas aos vendedores e clientes.
  - **olist_sellers_dataset** : Contém informações relativas a todos os vendedores que se cadastraram nesta empresa.

#### Quantidade de pedidos


In [0]:
%sql
SELECT 
    COUNT(*) AS total_pedidos_entregues
    FROM poc_datum.olist.order_silver

total_pedidos_entregues
99441


Databricks visualization. Run in Databricks to view.

#### Quantidade de pedidos por Status

In [0]:
%sql
WITH total_pedidos_entregues 
AS (
    SELECT 
    COUNT(*) AS total_pedidos_entregues
    FROM poc_datum.olist.order_silver
    WHERE order_status = 'delivered'
),
total_pedidos_n_entregue 
AS (
    SELECT 
    COUNT(*) AS total_pedidos_n_entregues
    FROM poc_datum.olist.order_silver
    WHERE order_status <> 'delivered'
)

SELECT 'Delivery' AS status, tpe.total_pedidos_entregues FROM  total_pedidos_entregues AS tpe
UNION
SELECT 'No Delivery' AS status, tpne.total_pedidos_n_entregues FROM  total_pedidos_n_entregue AS tpne



status,total_pedidos_entregues
Delivery,96478
No Delivery,2963


Databricks visualization. Run in Databricks to view.

In [0]:
total_pedidos_n_entregue = spark.sql("""
                                    SELECT 
                                        COUNT(*) AS total_pedidos_n_entregues
                                    FROM poc_datum.olist.order_silver
                                        WHERE order_status <> 'delivered'
                                    """).collect()[0][0]

total_pedidos = spark.sql("""
                            SELECT 
                                COUNT(*) AS total_pedidos
                            FROM poc_datum.olist.order_silver
                            """).collect()[0][0]

percent_no_delivery = total_pedidos_n_entregue / total_pedidos

print("Percentual de pedidos não entregues: {:.2%}".format(percent_no_delivery))

#Percentual de pedidos não entregues: 2.98%


1. **Eficiência nas Entregas**: A diferença entre as barras é bastante acentuada, com a barra de entregas muito maior que a de não entregues. Isso sugere que a maioria dos pedidos é concluída com sucesso.

2. **Pedidos Não Entregues**: Apesar que a quantidade seja significativamente menor (aprox. **2,98%**), ainda representa uma quantidade de pedidos que não chegaram ao destino final. É importante entender os motivos por trás desses casos para melhorar a eficácia da entrega.

In [0]:
%sql
WITH total_pedidos AS (
    SELECT 
    COUNT(*) AS total_geral_pedidos
    FROM poc_datum.olist.order_silver
    WHERE order_status <> 'delivered'
),

total_pedidos_n_entregue AS (
    SELECT 
    DISTINCT
    order_status,
    COUNT(*) AS total_pedidos
    FROM poc_datum.olist.order_silver
    WHERE order_status <> 'delivered'
    GROUP BY order_status
)

SELECT 
    s.order_status,
    s.total_pedidos,
    (s.total_pedidos * 100.0 / t.total_geral_pedidos) AS percentual
FROM 
    total_pedidos_n_entregue s,
    total_pedidos t
ORDER BY 
    s.total_pedidos DESC;

order_status,total_pedidos,percentual
shipped,1107,37.36078299021262
canceled,625,21.09348633142086
unavailable,609,20.55349308133648
invoiced,314,10.59736753290584
processing,301,10.15862301721228
created,5,0.16874789065137
approved,2,0.06749915626055


Databricks visualization. Run in Databricks to view.


1. **Maior Categoria**: A maior barra representa pedidos que foram enviados ("shipped"), indicando que esta é a etapa mais comum atualmente no status dos pedidos. Isso é esperado, já que em qualquer momento, muitos pedidos estarão em trânsito.

2. **Pedidos Cancelados**: A segunda maior barra indica uma quantidade significativa de pedidos cancelados. Isso pode sinalizar uma área que merece investigação, pois altas taxas de cancelamento podem indicar problemas no processo de compra, insatisfação do cliente ou problemas de estoque.

3. **Disponibilidade**: A terceira maior barra, "unavailable", sugere que muitos produtos não estão disponíveis. Isso pode ser um ponto de atenção para a gestão de estoque, podendo afetar a satisfação dos clientes e marketing negativo por falta de produtos.

4. **Processamento e Faturamento**: As barras para "processing" e "invoiced" mostram números menores. Isso aparentemente é normal, pois representam etapas transitórias rápidas no ciclo de vida de um pedido. Se esses números fossem anormalmente altos, poderia indicar gargalos no processamento de pedidos ou na faturamento da nota.

5. **Criação e Aprovação**: Existem muito poucos pedidos nas etapas iniciais de "created" e "approved". Isso pode indicar que os pedidos estão sendo processados rapidamente através dessas etapas iniciais, o que geralmente é um bom sinal de eficiência.

#### Evolução ao longo do tempo

In [0]:
%sql
WITH total_pedidos 
AS (
SELECT  
  date_format(ops.order_purchase_timestamp, 'yyyy-MM' ) AS mes_ano ,*
FROM 
  poc_datum.olist.order_silver AS ops )

SELECT
  mes_ano,
  count(*) as total_pedidos
FROM total_pedidos
GROUP BY mes_ano
ORDER BY 1 

mes_ano,total_pedidos
2016-09,4
2016-10,324
2016-12,1
2017-01,800
2017-02,1780
2017-03,2682
2017-04,2404
2017-05,3700
2017-06,3245
2017-07,4026


Databricks visualization. Run in Databricks to view.

### Premissa: Identificando Padrões de Compra
_Introduza uma regra mais complexa, como identificar padrões de comportamento de compra ao longo do tempo ou criar categorias personalizadas de produtos com base em determinados critérios._

In [0]:
%sql
WITH total_pedidos 
AS (
SELECT  
  date_format(ops.order_purchase_timestamp, 'yyyy-MM-dd' ) AS dia ,*
FROM 
  poc_datum.olist.order_silver AS ops )

SELECT
  dia,
  count(*) as total_pedidos
FROM total_pedidos
GROUP BY dia
ORDER BY 1 

dia,total_pedidos
2016-09-04,1
2016-09-05,1
2016-09-13,1
2016-09-15,1
2016-10-02,1
2016-10-03,8
2016-10-04,63
2016-10-05,47
2016-10-06,51
2016-10-07,46


Databricks visualization. Run in Databricks to view.

1. **Crescimento ao Longo do Tempo**: Existe uma tendência geral de crescimento no volume de pedidos ao longo do tempo, indicando um aumento na atividade de vendas ou na aquisição de clientes.

2. **Flutuações Sazonais**: Parece haver uma sazonalidade nas vendas, com picos e vales que podem corresponder a períodos específicos do ano, como feriados, eventos sazonais ou promoções.

3. **Pico de Vendas**: Houve um pico de vendas com mais de 1.000 pedidos em 24 de novembro de 2017, em pesquisas, nota-se que foi em função da Blackfriday.

4. **Queda Recente**: Há uma queda acentuada no número de pedidos no ponto mais recente do gráfico. Isso pode ser devido à dados incompletos para o mês mais recente.

In [0]:
%sql
SELECT DISTINCT
date_format(order_purchase_timestamp, 'HH') AS hora,
count(*) as total_pedidos
FROM poc_datum.olist.order_silver
GROUP BY ALL
ORDER BY 1

hora,total_pedidos
00,2394
01,1170
02,510
03,272
04,206
05,188
06,502
07,1231
08,2967
09,4785


Databricks visualization. Run in Databricks to view.

1. **Horário de Pico**: Há um pico evidente à partir das 8h, que é crescente até o horário de almoço (por volta das 11h até as 13h) e no meio da tarde se mantem até as 22h. Isso sugere que as pessoas podem estar fazendo compras durante o horário de trabalho ou aproveitando as pausas para realizar pedidos. Também pode indicar que o comportamento de compra online é consistente ao longo da tarde e início da noite, o que pode refletir o tempo livre que os consumidores têm após o trabalho para atividades de lazer, como compras online

2. **Horas da Madrugada**: As vendas caem significativamente após a meia-noite e começam a aumentar novamente após as 6h. Isso é consistente com os padrões de sono e atividade da maioria das pessoas.

3. **Manhã vs. Noite**: As vendas parecem ser mais altas à tarde e à noite em comparação com as horas da manhã. Isso sugere oportunidade em campanhas de marketing e promoções nesse período.

4. **Menor Atividade**: O período entre 3h e 5h da manhã tem o menor número de pedidos, o que é típico, considerando que a maioria das pessoas está dormindo.


### Modalidade de pagamento

In [0]:
%sql
SELECT DISTINCT 
  payment_type, 
  count(*) as total_pedidos 
FROM 
  poc_datum.olist.order_payments_silver
GROUP BY payment_type
ORDER BY 2 DESC

payment_type,total_pedidos
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


Databricks visualization. Run in Databricks to view.

1. **Cartão de Crédito**: É o método mais popular, com a maior barra no gráfico, indicando que a grande maioria dos clientes prefere pagar com cartão de crédito. Isso pode refletir a conveniência do pagamento a prazo, a segurança, a acumulação de pontos de recompensa, ou a possibilidade de financiamento das compras.

2. **Boleto**: O segundo método mais comum é o boleto bancário. Esta preferência pode estar ligada à popularidade do boleto no Brasil como método de pagamento confiável e amplamente aceito, ou ao fato de ser uma opção para quem não possui cartão de crédito ou não deseja usá-lo online.

3. **Voucher**: Os vouchers são a terceira forma de pagamento mais utilizada, o que pode indicar promoções, programas de fidelidade ou a utilização de créditos de devoluções anteriores.

4. **Cartão de Débito**: O cartão de débito aparece em menor medida, o que pode sugerir uma preferência menor por pagamentos imediatos ou talvez menos promoções ou benefícios associados a esta forma de pagamento em comparação com cartões de crédito.

5. **Não Definido**: Há um número muito pequeno de transações categorizadas como "não definido", que poderia ser devido a erros de processamento ou categorização.